In [1]:
import numpy as np
import random
from random import randint
from gurobipy import *

In [2]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

In [3]:
# Product Demand
demand = np.random.randint(0,20,(Products,Market))

In [4]:
# Cost of opening
f_i = [20, 25]
f_j = [15, 5, 10]

In [5]:
# Unit cost of manufacturing product 
Manufacturing_costs = np.random.uniform(0,5, (Manufacturing_plants,Products))

In [6]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,5, (Products, Manufacturing_plants, Distribution))

In [7]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,5, (Products, Distribution, Market))

In [8]:
# Plant Capacities
Capacities_i = np.random.randint(30,50,(Manufacturing_plants))
Capacities_j = np.random.randint(30,50,(Distribution))
Capacities_l = np.zeros((Products,Outsourced))
np.fill_diagonal(Capacities_l, random.randint(15,20))

In [9]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
Supplier_cost = np.zeros((Products, Outsourced))
np.fill_diagonal(Supplier_cost, random.uniform(10,15))

In [10]:
# Cost of transporting product m from supplier to j
T_O_DC = np.zeros((Products, Outsourced, Distribution))
T_O_DC[0][0] = np.random.uniform(5,10,(Distribution))
T_O_DC[1][1] = np.random.uniform(5,10,(Distribution))

In [11]:
# Cost of shipping product m from supplier to k
T_O_MZ = np.zeros((Products, Outsourced, Market))
T_O_MZ[0][0] = np.random.uniform(5,10,(Market))
T_O_MZ[1][1] = np.random.uniform(5,10,(Market))

In [12]:
# Product volume 
volume = np.random.randint(2,5,(Products))

In [13]:
# unit cost of lost sales 
lost_sales = np.random.randint(75,100,(Market,Products))

In [14]:
## Model

In [15]:
grbModel = Model('synthetic')

Using license file C:\Users\Devika Kabe\gurobi.lic
Academic license - for non-commercial use only


In [16]:
a_i = np.ones(Manufacturing_plants)
b_j = np.ones(Distribution)

In [17]:
# Model Variables

x_i = grbModel.addVars(range(Manufacturing_plants), vtype = GRB.BINARY) # opening manufacturing plant
x_j = grbModel.addVars(range(Distribution), vtype = GRB.BINARY) # opening DC
U_km = grbModel.addVars(range(Market), range(Products), vtype = GRB.INTEGER) # quantity lost sales
V_lm = grbModel.addVars(range(Products), range(Outsourced), vtype = GRB.INTEGER) # quantity products purchased from outsourcing
Q_im = grbModel.addVars(range(Products), range(Manufacturing_plants), vtype = GRB.INTEGER) # quantity produced
Y_ijm = grbModel.addVars(range(Products), range(Manufacturing_plants), range(Distribution), vtype = GRB.INTEGER) # shipping i -> j
Z_jkm = grbModel.addVars(range(Products), range(Distribution), range(Market), vtype = GRB.INTEGER) # shipping j -> k
T_ljm = grbModel.addVars(range(Products), range(Outsourced), range(Distribution), vtype = GRB.INTEGER) # shipping l -> j
T_lkm = grbModel.addVars(range(Products), range(Outsourced), range(Market), vtype = GRB.INTEGER) # shipping l -> k


In [18]:
# Model Constraints
# Network Flow
#grbModel.addConstr(quicksum(x_i[i] for i in range(Manufacturing_plants)) >= 1)
#grbModel.addConstr(quicksum(x_j[j] for i in range(Distribution)) >= 1)

grbModel.addConstrs(quicksum(Y_ijm[m,i,j] for j in range(Distribution)) == Q_im[m,i] 
             for i in range(Manufacturing_plants) for m in range(Products))

grbModel.addConstrs(quicksum(Z_jkm[m,j,k] for k in range(Market)) == 
                      (quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) + 
                       quicksum(T_ljm[m,l,j] for l in range(Outsourced))) for j in range(Distribution) 
                      for m in range(Products))

grbModel.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) + quicksum(T_lkm[m,l,k] for l in range(Outsourced))) ==
                      (demand[m][k] - U_km[k,m]) 
                    for k in range(Market) for m in range(Products))


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>}

In [19]:
for m in range(Products):
    for l in range(Outsourced):
        if l != m:
            grbModel.addConstr(V_lm[m,l] == 0)

In [20]:
for m in range(Products):
    for l in range(Outsourced):
        for j in range(Distribution):
            if l != m:
                grbModel.addConstr(T_ljm[m,l,j] == 0)

In [21]:
for m in range(Products):
    for l in range(Outsourced):
        for k in range(Market):
            if l != m:
                grbModel.addConstr(T_lkm[m,l,k] == 0)

In [22]:
# Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
grbModel.addConstrs(V_lm[m,l] == quicksum(T_ljm[m,l,j] for j in range(Distribution)) + quicksum(T_lkm[m,l,k] for k in range(Market))
            for m in range(Products) for l in range(Outsourced))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>}

In [23]:
# Capacity Constraints
grbModel.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= a_i[i]*Capacities_i[i]*x_i[i] for i in range(Manufacturing_plants))
grbModel.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
            quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= b_j[j]*Capacities_j[j]*x_j[j] 
            for j in range(Distribution))
grbModel.addConstrs(quicksum(V_lm[m,l] for m in range(Products)) <= np.sum(Capacities_l[l]) for l in range(Outsourced))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [24]:
# Objective
grb_expr = LinExpr()

# Cost of opening
OC_1 = 0
OC_2 = 0
for i in range(Manufacturing_plants):
    OC_1 += f_i[i]*x_i[i]
for j in range(Distribution):
    OC_2 += f_j[j]*x_j[j]
    
#grb_expr += OC_1 + OC_2
ship_1 = 0
ship_2 = 0
ship_3 = 0
ship_4 = 0

# Shipment
for i in range(Manufacturing_plants):
    for j in range(Distribution):
        for m in range(Products):
            ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]
            
for j in range(Distribution):
    for k in range(Market):
        for m in range(Products):
            ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

for l in range(Outsourced):
    for j in range(Distribution):
        for m in range(Products):
            ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

for l in range(Outsourced):
    for k in range(Market):
        for m in range(Products):
            ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]

#grb_expr += ship_1 + ship_2 + ship_3 + ship_4

# Production
pr_cost = 0
for i in range(Manufacturing_plants):
    for m in range(Products):
        pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]

#grb_expr += pr_cost
# Buying from outsource cost
b_cost = 0
for l in range(Outsourced):
    for m in range(Products):
        b_cost += Supplier_cost[m][l]*V_lm[m,l]
#grb_expr += b_cost

# Lost Sales
l_cost = 0
for k in range(Market):
    for m in range(Products):
        l_cost += lost_sales[k][m]*U_km[k,m]

grb_expr += OC_1 + OC_2 + ship_1 + ship_2 + ship_3 + ship_4 + pr_cost + b_cost + l_cost
        

In [25]:
grbModel.setObjective(grb_expr, GRB.MINIMIZE)

In [26]:
grbModel.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 45 rows, 85 columns and 197 nonzeros
Model fingerprint: 0x974f6840
Variable types: 0 continuous, 85 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e-03, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 9037.0000000
Presolve removed 21 rows and 27 columns
Presolve time: 0.00s
Presolved: 24 rows, 58 columns, 129 nonzeros
Variable types: 0 continuous, 58 integer (5 binary)

Root relaxation: objective 3.271935e+03, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3271.93468    0    6 9037.00000 3271.93468  63.8%     -    0s
H    0     0                    3305.2779985 3271.93468  1.01%     -    0s
     0     0 3285.08780    0    1 3305.27800 3285.08780  0.61%   

In [27]:
v_val_x_i = grbModel.getAttr('x', x_i)
v_val_x_j = grbModel.getAttr('x', x_j)
v_val_U_km = grbModel.getAttr('x', U_km)
v_val_V_lm = grbModel.getAttr('x', V_lm)
v_val_Q_im = grbModel.getAttr('x', Q_im)
v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
v_val_T_ljm = grbModel.getAttr('x', T_ljm)
v_val_T_lkm = grbModel.getAttr('x', T_lkm)

In [28]:
obj = grbModel.getObjective()
print("obj val: ", obj.getValue())
print("Open Manufacturing Plants: ", np.sum(v_val_x_i.values()))
print("Open Distribution Centres: ", np.sum(v_val_x_j.values()))
print("Total purchased from Outsourcing: ", np.sum(v_val_V_lm.values()))
print("Total produced in house: ", np.sum(v_val_Q_im.values()))
print("Total lost sales: ", np.sum(v_val_U_km.values()))
print("Total shipped from outsourced facility to DCs: ", np.sum(v_val_T_ljm.values()))
print("Total shipped directly to market zone: ", np.sum(v_val_T_lkm.values()))

obj val:  3286.754471399328
Open Manufacturing Plants:  2.0
Open Distribution Centres:  3.0
Total purchased from Outsourcing:  40.0
Total produced in house:  37.0
Total lost sales:  26.0
Total shipped from outsourced facility to DCs:  0.0
Total shipped directly to market zone:  40.0


In [38]:
grb_expr

<gurobi.LinExpr: 20.0 C0 + 25.0 C1 + 15.0 C2 + 5.0 C3 + 10.0 C4 + 3.4185278555192022 C21 + 1.7851155801816143 C27 + 4.539256650776101 C22 + 0.00638617332535063 C28 + 1.5999512371974167 C23 + 1.5018591163268735 C29 + 0.601744574829306 C24 + 3.6002327976127644 C30 + 2.3694089065041526 C25 + 4.473630906639036 C31 + 4.98299871203023 C26 + 3.619290766399147 C32 + 4.585934537208878 C33 + 4.490980424943651 C45 + 2.7691293146801357 C34 + 2.6752705814178563 C46 + 3.692331742757627 C35 + 3.7786984908878676 C47 + 1.9292579912147356 C36 + 3.9155811264342373 C48 + 4.01820420955375 C37 + 0.4309855158271969 C49 + 4.304313008473909 C38 + 4.130447266459018 C50 + 4.662783613308868 C39 + 2.114389901057357 C51 + 1.9721653416290474 C40 + 1.4569173913928495 C52 + 3.032164074428485 C41 + 3.789946014847061 C53 + 4.258158632539249 C42 + 0.366355246115061 C54 + 0.7469991301297502 C43 + 3.7319569242666772 C55 + 4.215405466005334 C44 + 1.365506655502649 C56 + 6.617754832186695 C57 + 0.0 C63 + 9.915252225631715 C5

In [39]:
v_val_U_km

{(0, 0): 0.0,
 (0, 1): 0.0,
 (1, 0): 4.0,
 (1, 1): 12.0,
 (2, 0): 0.0,
 (2, 1): 3.0,
 (3, 0): 0.0,
 (3, 1): 7.0}